In [55]:
import pandas as pd
import numpy as np
from functools import reduce

In [56]:
moaks_sq03 = pd.read_csv('/home/anastasis/EMC_Thesis/OAI Data/OAICompleteData_ASCII/kMRI_SQ_MOAKS_BICL03.txt', sep="|")

In [57]:
print(moaks_sq03.info(),'\n',moaks_sq03.iloc[:,-3])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2107 entries, 0 to 2106
Columns: 121 entries, ID to V03MTCMNTS
dtypes: float64(16), int64(1), object(104)
memory usage: 1.9+ MB
None 
 0       .: Missing Form/Incomplete Workbook
1       .: Missing Form/Incomplete Workbook
2       .: Missing Form/Incomplete Workbook
3       .: Missing Form/Incomplete Workbook
4       .: Missing Form/Incomplete Workbook
                       ...                 
2102    .: Missing Form/Incomplete Workbook
2103    .: Missing Form/Incomplete Workbook
2104    .: Missing Form/Incomplete Workbook
2105    .: Missing Form/Incomplete Workbook
2106    .: Missing Form/Incomplete Workbook
Name: V03MITBSIG, Length: 2107, dtype: object


# NOW WE DELETE ALL THE COMMENTS OF THE INITIAL DATAFRAME AND CONVERT THE VALUES TO NUMBERS

In [58]:
# Firstly we replace all the comments
moaks_without_commments = moaks_sq03.replace(r'(\:).*$',r'\1',regex=True)
# Secondly we replace all the '.:' and the ':' bullets with space and remove 'R' from Reader Column
moaks_without_commments_bullets = moaks_without_commments.replace({'\.\:':'',':':''},regex=True)
# Thirdly we remove the Comments and Technical Considerations Columns from the whole Dataset
moaks_without_commments_bullets = moaks_without_commments_bullets.drop(columns=['V03MCMNTS','V03MTCMNTS'])
print(moaks_without_commments_bullets.READPRJ.value_counts())

65     870
22     600
63E    297
63A    103
63F     80
63C     62
63B     60
63D     35
Name: READPRJ, dtype: int64


In [59]:
# WE KEEP ONLY THE SUBJECTS THAT ARE PRESENT IN PROJECT 65 AND NOT IN PROJECT 22

moaks_without_commments_bullets_prj22 = moaks_without_commments_bullets.loc[moaks_without_commments_bullets['READPRJ']=='22']
print(moaks_without_commments_bullets_prj22.shape[0])

moaks_without_commments_bullets_prj63B = moaks_without_commments_bullets.loc[moaks_without_commments_bullets['READPRJ']=='63B']
print('Number of subjects with OA at 24 months prj63: {}'.format(moaks_without_commments_bullets_prj63B.shape[0]))
moaks_without_commments_bullets_prj63B_unique_ID = pd.merge(moaks_without_commments_bullets_prj63B,moaks_without_commments_bullets_prj22,on=['ID','SIDE'],how='outer',indicator=True)
moaks_without_commments_bullets_prj63B_unique_ID = moaks_without_commments_bullets_prj63B_unique_ID.loc[(moaks_without_commments_bullets_prj63B_unique_ID._merge=='left_only'),['ID','SIDE']]
moaks_without_commments_bullets_prj63B_unique_info = moaks_without_commments_bullets_prj63B.merge(moaks_without_commments_bullets_prj63B_unique_ID,on=['ID','SIDE'],how='inner')
moaks_without_commments_bullets_prj63B_unique_info = moaks_without_commments_bullets_prj63B_unique_info.drop_duplicates(subset=['ID','SIDE'])
print('Number of subjects with OA at 24 months unique to prj63: {}'.format(moaks_without_commments_bullets_prj63B_unique_info.shape[0]))


600
Number of subjects with OA at 24 months prj63: 60
Number of subjects with OA at 24 months unique to prj63: 50


In [60]:
# THEN WE CONVERT ALL THE DF VALUES TO FLOAT AND THE ID AND SIDE TO INT
# THEN WE CONVERT ALL THE DF VALUES TO FLOAT AND THE ID AND SIDE TO INT
moaks_without_commments_bullets_prj63B_unique_info.replace(to_replace={'63B':'63.1'},inplace=True)
moaks_without_c_b_numeric_63B =  moaks_without_commments_bullets_prj63B_unique_info.apply(pd.to_numeric,downcast='float')
moaks_without_c_b_numeric_63B[['ID','SIDE']] = moaks_without_c_b_numeric_63B[['ID','SIDE']].astype('int')
print(moaks_without_c_b_numeric_63B.info())
print('Number of NaN values are : ',moaks_without_c_b_numeric_63B.isna().sum().sum())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Columns: 119 entries, ID to V03MITBSIG
dtypes: float32(117), int64(2)
memory usage: 24.0 KB
None
Number of NaN values are :  3800


In [61]:
print('Number of NaN values in the initial dataframe: {}'.format(moaks_without_c_b_numeric_63B.loc[:,:].isna().sum().to_list()))

Number of NaN values in the initial dataframe: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 0, 50, 50, 0, 0, 0, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


In [62]:
moaks_without_c_b_nan_numeric = moaks_without_c_b_numeric_63B.fillna(-1.0)
moaks_without_c_b_nan_numeric.info()
print('Number of NaN values in the Dataframe: {}'.format(moaks_without_c_b_nan_numeric.isna().sum().to_list()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Columns: 119 entries, ID to V03MITBSIG
dtypes: float32(117), int64(2)
memory usage: 24.0 KB
Number of NaN values in the Dataframe: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [63]:
# So the clean version of the moaks_sq03 is the below: 
moaks_grades = moaks_without_c_b_nan_numeric
print(moaks_grades.isna().sum().to_list())
moaks_grades.to_csv('moaks_sq03_initial_numeric_df.csv')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# NOW WE LOAD THE RADIOGRAPH INFORMATION DATABASE IN ORDER TO ACQUIRE THE XR OSTEOPHYTES


In [64]:
kxr_sq_bu03= pd.read_csv('/home/anastasis/EMC_Thesis/OAI Data/OAICompleteData_ASCII/kxr_sq_bu03.txt', sep="|")
KL_prj15 = kxr_sq_bu03[kxr_sq_bu03['readprj'] == 15].copy()
KL_prj15_osteophytes = KL_prj15.loc[:,['ID','SIDE','V03XROSFL','V03XROSFM','V03XROSTL','V03XROSTM','V03XRKL']]

#now we create the df that contains both MOAKS grades and XR Osteophytes
common_sq_kxr_03 = pd.merge(moaks_grades,KL_prj15_osteophytes,on=['ID','SIDE'],how='inner')
common_sq_kxr_03.fillna(0.0,inplace=True)
common_sq_kxr_03

,ID,SIDE,READPRJ,VERSION,V03MCMPM,V03MCMPL,V03MCMFMA,V03MCMFLA,V03MCMFMP,V03MCMFLP,...,V03MPPBUR,V03MSYIC,V03MEFFWK,V03MPOPCYS,V03MITBSIG,V03XROSFL,V03XROSFM,V03XROSTL,V03XROSTM,V03XRKL
0,9031930,1,63.099998,3.2,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,1.0,3.0
1,9039627,1,63.099998,3.2,0.0,0.0,0.0,2.2,1.1,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,1.0,1.0,2.0
2,9055038,1,63.099998,3.2,2.0,2.2,2.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,2.0
3,9056978,1,63.099998,3.2,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,2.0
4,9057479,1,63.099998,3.2,2.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,2.0
5,9057479,2,63.099998,3.2,2.0,0.0,2.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,2.0
6,9064631,2,63.099998,3.2,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,2.0
7,9081858,2,63.099998,3.2,2.1,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,2.0
8,9082137,1,63.099998,3.2,0.0,0.0,0.0,2.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0,2.0
9,9086407,1,63.099998,3.2,2.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,1.0,2.0


# NOW WE FIND THE SUBJECTS THAT FULFILL THE OA FEATURES FOR THE WHOLE KNEE JOINT

In [65]:
# FRIST WE CONSTRUCT A NEW DATAFRAME WITH ALL THE OA FEATURES AS COLUMNS
# The 'AnyTCL' column is an assistive variable that indicates as 1.0
# the subjects with at least on variable of thickness cartilage loss >=1.0
# and as 0.0 the subjects that DO NOT have any thickness cartilage loss

moaks_OA_features = pd.DataFrame(
    columns=['ID','SIDE','XR_Osteophytes','FullTCL','AnyTCL',
             'PartialTCL','BML', 'Meniscus_Degradation'])
# the ID and SIDE columns are integers
moaks_OA_features[['ID','SIDE']] = common_sq_kxr_03[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
moaks_OA_features.iloc[:,2:] = np.nan

print('Type of ID and SIDE columns {} and type of the rest of the columns {}'
    .format(type(moaks_OA_features.iloc[0,0]),type(moaks_OA_features.iloc[0,2])))


Type of ID and SIDE columns <class 'numpy.int64'> and type of the rest of the columns <class 'numpy.float64'>


In [66]:
moaks_OA_features

,ID,SIDE,XR_Osteophytes,FullTCL,AnyTCL,PartialTCL,BML,Meniscus_Degradation
0,9031930,1,NaN,NaN,NaN,NaN,NaN,NaN
1,9039627,1,NaN,NaN,NaN,NaN,NaN,NaN
2,9055038,1,NaN,NaN,NaN,NaN,NaN,NaN
3,9056978,1,NaN,NaN,NaN,NaN,NaN,NaN
4,9057479,1,NaN,NaN,NaN,NaN,NaN,NaN
5,9057479,2,NaN,NaN,NaN,NaN,NaN,NaN
6,9064631,2,NaN,NaN,NaN,NaN,NaN,NaN
7,9081858,2,NaN,NaN,NaN,NaN,NaN,NaN
8,9082137,1,NaN,NaN,NaN,NaN,NaN,NaN
9,9086407,1,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Then we fill the FullTCL column with all the subjects with 
# at least one cartilage thickness loss variable larger than 3.0
# and the FullTCL cell value is 1.0
moaks_OA_features.loc[((common_sq_kxr_03['V03MCMFLA'] >= 3.0) | (common_sq_kxr_03['V03MCMFLP'] >= 3.0) |
                (common_sq_kxr_03['V03MCMFLC'] >= 3.0) | (common_sq_kxr_03['V03MCMTLA'] >= 3.0) |
                (common_sq_kxr_03['V03MCMTLP'] >= 3.0) | (common_sq_kxr_03['V03MCMTLC'] >= 3.0) |
                (common_sq_kxr_03['V03MCMFMA'] >= 3.0) | (common_sq_kxr_03['V03MCMFMP'] >= 3.0) |
                (common_sq_kxr_03['V03MCMFMC'] >= 3.0) | (common_sq_kxr_03['V03MCMTMA'] >= 3.0) |
                (common_sq_kxr_03['V03MCMTMP'] >= 3.0) | (common_sq_kxr_03['V03MCMTMC'] >= 3.0)),'FullTCL']=1.0

In [68]:
print('Number of subjects WITH Full CTL : {}'.format((moaks_OA_features['FullTCL']==1.0).sum()))


Number of subjects WITH Full CTL : 6


In [69]:
# Then we fill the rest of the values of Full_CTL column with 0.0 values instead of NaN
moaks_OA_features.loc[((common_sq_kxr_03['V03MCMFLA'] < 3.0) &  (common_sq_kxr_03['V03MCMFLA'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFLP'] < 3.0) & (common_sq_kxr_03['V03MCMFLP'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFLC'] < 3.0) & (common_sq_kxr_03['V03MCMFLC'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTLA'] < 3.0) & (common_sq_kxr_03['V03MCMTLA'] >= 0.0) &
                (common_sq_kxr_03['V03MCMTLP'] < 3.0) & (common_sq_kxr_03['V03MCMTLP'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTLC'] < 3.0) & (common_sq_kxr_03['V03MCMTLC'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFMA'] < 3.0) & (common_sq_kxr_03['V03MCMFMA'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMFMP'] < 3.0) & (common_sq_kxr_03['V03MCMFMP'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFMC'] < 3.0) & (common_sq_kxr_03['V03MCMFMC'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTMA'] < 3.0) & (common_sq_kxr_03['V03MCMTMA'] >= 0.0) &
                (common_sq_kxr_03['V03MCMTMP'] < 3.0) & (common_sq_kxr_03['V03MCMTMP'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTMC'] < 3.0) & (common_sq_kxr_03['V03MCMTMC'] >= 0.0)),'FullTCL']=0.0
print(moaks_OA_features['FullTCL'].value_counts())


0.0    44
1.0     6
Name: FullTCL, dtype: int64


In [70]:
# now the column AnyTCL is filled based on the initial Thickness Cartilage Loss variables
# if at least one variable value is larger than 1.0 then it has either Full or Partial Thickness Cartilage Loss
# and the AnyTCL cell value is 1.0

moaks_OA_features.loc[((common_sq_kxr_03['V03MCMFLA'] >= 1.0) | (common_sq_kxr_03['V03MCMFLP'] >= 1.0) |
                (common_sq_kxr_03['V03MCMFLC'] >= 1.0) | (common_sq_kxr_03['V03MCMTLA'] >= 1.0) |
                (common_sq_kxr_03['V03MCMTLP'] >= 1.0) | (common_sq_kxr_03['V03MCMTLC'] >= 1.0) |
                (common_sq_kxr_03['V03MCMFMA'] >= 1.0) | (common_sq_kxr_03['V03MCMFMP'] >= 1.0) |
                (common_sq_kxr_03['V03MCMFMC'] >= 1.0) | (common_sq_kxr_03['V03MCMTMA'] >= 1.0) |
                (common_sq_kxr_03['V03MCMTMP'] >= 1.0) | (common_sq_kxr_03['V03MCMTMC'] >= 1.0)),'AnyTCL']=1.0



In [71]:
# if all Thickness Cartilage Loss variables are less than 1.0 then the subject has no Thickness Cartilage Loss
# and the AnyTCL cell value is 0.0
moaks_OA_features.loc[((common_sq_kxr_03['V03MCMFLA'] < 1.0) & (common_sq_kxr_03['V03MCMFLA'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMFLP'] < 1.0) & (common_sq_kxr_03['V03MCMFLP'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFLC'] < 1.0) & (common_sq_kxr_03['V03MCMFLC'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTLA'] < 1.0) & (common_sq_kxr_03['V03MCMTLA'] >= 0.0) &
                (common_sq_kxr_03['V03MCMTLP'] < 1.0) & (common_sq_kxr_03['V03MCMTLP'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTLC'] < 1.0) & (common_sq_kxr_03['V03MCMTLC'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFMA'] < 1.0) & (common_sq_kxr_03['V03MCMFMA'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMFMP'] < 1.0) & (common_sq_kxr_03['V03MCMFMP'] >= 0.0) &
                (common_sq_kxr_03['V03MCMFMC'] < 1.0) & (common_sq_kxr_03['V03MCMFMC'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTMA'] < 1.0) & (common_sq_kxr_03['V03MCMTMA'] >= 0.0) &
                (common_sq_kxr_03['V03MCMTMP'] < 1.0) & (common_sq_kxr_03['V03MCMTMP'] >= 0.0) & 
                (common_sq_kxr_03['V03MCMTMC'] < 1.0) & (common_sq_kxr_03['V03MCMTMC'] >= 0.0)),'AnyTCL']=0.0
print('Number of Subjects WITHOUT Any Cartilage Thickness Loss: {}'.
        format((moaks_OA_features['AnyTCL']==0.0).sum()))


Number of Subjects WITHOUT Any Cartilage Thickness Loss: 2


In [72]:
print(moaks_OA_features.AnyTCL.value_counts())

1.0    48
0.0     2
Name: AnyTCL, dtype: int64


In [73]:
# Now we fill the PartialTCL column initially with the values of the AnyTCL column
# Where the FullTCL column is 1.0 then the PartialTCL is 0.0
# So if the PartialTCL is 1.0 then the Thickness Cartilage Loss variables are between 1.0 and 3.0

moaks_OA_features['PartialTCL'] = moaks_OA_features['AnyTCL']
moaks_OA_features.loc[(moaks_OA_features['FullTCL']==1),'PartialTCL']=0.0
print('Number of Subjects with Partial Thickness Cartilage Loss: {}'.
        format((moaks_OA_features['PartialTCL']==1.0).sum()))

Number of Subjects with Partial Thickness Cartilage Loss: 42


In [74]:
(common_sq_kxr_03.loc[:,['V03MCMFLA','V03MCMFLP','V03MCMFLP','V03MCMTLA','V03MCMTLP','V03MCMTLC'
                    ,'V03MCMFMA','V03MCMFMP','V03MCMFMP','V03MCMTMA','V03MCMTMP','V03MCMTMC']]==-1.0).sum()

V03MCMFLA    0
V03MCMFLP    0
V03MCMFLP    0
V03MCMTLA    0
V03MCMTLP    0
V03MCMTLC    0
V03MCMFMA    0
V03MCMFMP    0
V03MCMFMP    0
V03MCMTMA    0
V03MCMTMP    0
V03MCMTMC    0
dtype: int64

In [75]:
# NOW WE FILL THE XR OSTEOPHYTES COLUMN

moaks_OA_features.loc[((common_sq_kxr_03['V03XROSFL'] >= 1.0) | 
                (common_sq_kxr_03['V03XROSFM'] >= 1.0) | 
                (common_sq_kxr_03['V03XROSTL'] >= 1.0) | 
                (common_sq_kxr_03['V03XROSTM'] >= 1.0) ),'XR_Osteophytes'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_03['V03XROSFL'] < 1.0) & (common_sq_kxr_03['V03XROSFL'] >= 0.0) & 
                (common_sq_kxr_03['V03XROSFM'] < 1.0) & (common_sq_kxr_03['V03XROSFM'] >= 0.0) & 
                (common_sq_kxr_03['V03XROSTL'] < 1.0) & (common_sq_kxr_03['V03XROSTL'] >= 0.0) & 
                (common_sq_kxr_03['V03XROSTM'] < 1.0) & (common_sq_kxr_03['V03XROSTM'] >= 0.0)),'XR_Osteophytes'] = 0.0

print('With XR Osteophytes->1.0, Without->0.0 \n{}'.format(moaks_OA_features.XR_Osteophytes.value_counts(dropna=False)))


With XR Osteophytes->1.0, Without->0.0 
1.0    48
0.0     2
Name: XR_Osteophytes, dtype: int64


In [76]:
(common_sq_kxr_03.loc[:,['V03XROSFL','V03XROSFM','V03XROSTL','V03XROSTM']]==-1.0).sum()

V03XROSFL    0
V03XROSFM    0
V03XROSTL    0
V03XROSTM    0
dtype: int64

In [77]:
# NOW WE FILL THE BML COLUMN

moaks_OA_features.loc[((common_sq_kxr_03['V03MBMSFLC'] >= 1.0) | (common_sq_kxr_03['V03MBMSFLP'] >= 1.0) |
                (common_sq_kxr_03['V03MBMSTLA'] >= 1.0) | (common_sq_kxr_03['V03MBMSTLC'] >= 1.0) |
                (common_sq_kxr_03['V03MBMSTLP'] >= 1.0) | (common_sq_kxr_03['V03MBMSFMC'] >= 1.0) | 
                (common_sq_kxr_03['V03MBMSFMP'] >= 1.0) | (common_sq_kxr_03['V03MBMSTMA'] >= 1.0) | 
                (common_sq_kxr_03['V03MBMSTMC'] >= 1.0) | (common_sq_kxr_03['V03MBMSTMP'] >= 1.0)),'BML'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_03['V03MBMSFLC'] < 1.0) & (common_sq_kxr_03['V03MBMSFLC'] >= 0.0) &
                (common_sq_kxr_03['V03MBMSFLP'] < 1.0) & (common_sq_kxr_03['V03MBMSFLP'] >= 0.0) &
                (common_sq_kxr_03['V03MBMSTLA'] < 1.0) & (common_sq_kxr_03['V03MBMSTLA'] >= 0.0) &
                (common_sq_kxr_03['V03MBMSTLC'] < 1.0) & (common_sq_kxr_03['V03MBMSTLC'] >= 0.0) &
                (common_sq_kxr_03['V03MBMSTLP'] < 1.0) & (common_sq_kxr_03['V03MBMSTLP'] >= 0.0) & 
                (common_sq_kxr_03['V03MBMSFMC'] < 1.0) & (common_sq_kxr_03['V03MBMSFMC'] >= 0.0) & 
                (common_sq_kxr_03['V03MBMSFMP'] < 1.0) & (common_sq_kxr_03['V03MBMSFMP'] >= 0.0) & 
                (common_sq_kxr_03['V03MBMSTMA'] < 1.0) & (common_sq_kxr_03['V03MBMSTMA'] >= 0.0) & 
                (common_sq_kxr_03['V03MBMSTMC'] < 1.0) & (common_sq_kxr_03['V03MBMSTMC'] >= 0.0) & 
                (common_sq_kxr_03['V03MBMSTMP'] < 1.0) & (common_sq_kxr_03['V03MBMSTMP'] >= 0.0)),'BML'] = 0.0
print('With BML->1.0, Without->0.0 \n{}'.format(moaks_OA_features.BML.value_counts(dropna=False)))


With BML->1.0, Without->0.0 
0.0    28
1.0    20
NaN     2
Name: BML, dtype: int64


In [78]:
(common_sq_kxr_03.loc[:,['V03MBMSFLC','V03MBMSFLP','V03MBMSTLA','V03MBMSTLC','V03MBMSTLP',
                    'V03MBMSFMC','V03MBMSFMP','V03MBMSTMA','V03MBMSTMC','V03MBMSTMP']]==-1.0).sum()

V03MBMSFLC    0
V03MBMSFLP    0
V03MBMSTLA    0
V03MBMSTLC    0
V03MBMSTLP    0
V03MBMSFMC    0
V03MBMSFMP    0
V03MBMSTMA    0
V03MBMSTMC    0
V03MBMSTMP    0
dtype: int64

In [79]:
# AND NOW WE FILL THE MENISCUS DEGRADATION COLUMN

moaks_OA_features.loc[((common_sq_kxr_03['V03MMTLA'] >= 2.0) | (common_sq_kxr_03['V03MMTLB'] >= 2.0) |
                (common_sq_kxr_03['V03MMTLP'] >= 2.0) | (common_sq_kxr_03['V03MMRTL'] >= 2.0) |
                (common_sq_kxr_03['V03MMTMA'] >= 2.0) | (common_sq_kxr_03['V03MMTMB'] >= 2.0) |
                (common_sq_kxr_03['V03MMTMP'] >= 2.0) | (common_sq_kxr_03['V03MMRTM'] >= 2.0) |
                (common_sq_kxr_03['V03MMXLL'] >= 1.0) | (common_sq_kxr_03['V03MMXLA'] >= 1.0) |
                (common_sq_kxr_03['V03MMXMM'] >= 1.0) | (common_sq_kxr_03['V03MMXMA'] >= 1.0)),'Meniscus_Degradation'] = 1.0

moaks_OA_features.loc[((common_sq_kxr_03['V03MMTLA'] < 2.0) & (common_sq_kxr_03['V03MMTLA'] >= 0.0) & 
                (common_sq_kxr_03['V03MMTLB'] < 2.0) & (common_sq_kxr_03['V03MMTLB'] >= 0.0) &
                (common_sq_kxr_03['V03MMTLP'] < 2.0) & (common_sq_kxr_03['V03MMTLP'] >= 0.0) & 
                (common_sq_kxr_03['V03MMRTL'] < 2.0) & (common_sq_kxr_03['V03MMRTL'] >= 0.0) &
                (common_sq_kxr_03['V03MMTMA'] < 2.0) & (common_sq_kxr_03['V03MMTMA'] >= 0.0) & 
                (common_sq_kxr_03['V03MMTMB'] < 2.0) & (common_sq_kxr_03['V03MMTMB'] >= 0.0) &
                (common_sq_kxr_03['V03MMTMP'] < 2.0) & (common_sq_kxr_03['V03MMTMP'] >= 0.0) & 
                (common_sq_kxr_03['V03MMRTM'] < 2.0) & (common_sq_kxr_03['V03MMRTM'] >= 0.0) &
                (common_sq_kxr_03['V03MMXLL'] < 1.0) & (common_sq_kxr_03['V03MMXLL'] >= 0.0) & 
                (common_sq_kxr_03['V03MMXLA'] < 1.0) & (common_sq_kxr_03['V03MMXLA'] >= 0.0) &
                (common_sq_kxr_03['V03MMXMM'] < 1.0) & (common_sq_kxr_03['V03MMXMM'] >= 0.0) & 
                (common_sq_kxr_03['V03MMXMA'] < 1.0) & (common_sq_kxr_03['V03MMXMA'] >= 0.0)),'Meniscus_Degradation'] = 0.0

print('With Meniscus Degradation ->1.0, Without->0.0 \n{}'.format(moaks_OA_features.Meniscus_Degradation.value_counts(dropna=False)))

# print('Number of Subjects WITH Meniscus Degradation: {}'.
#         format((moaks_OA_features['Meniscus_Degradation']==1.0).sum()))

# print('Number of Subjects WITHOUT Meniscus Degradation: {}'.
#         format((moaks_OA_features['Meniscus_Degradation']==0.0).sum()))

With Meniscus Degradation ->1.0, Without->0.0 
1.0    40
NaN    10
Name: Meniscus_Degradation, dtype: int64


In [80]:
(common_sq_kxr_03.loc[:,['V03MMTLA','V03MMTLB','V03MMTLP','V03MMRTL','V03MMTMA','V03MMTMB','V03MMTMP','V03MMRTM',
            'V03MMXLL','V03MMXLA','V03MMXMM','V03MMXMA']]==-1.0).sum()

V03MMTLA     0
V03MMTLB     0
V03MMTLP     0
V03MMRTL     0
V03MMTMA     0
V03MMTMB     0
V03MMTMP     0
V03MMRTM     0
V03MMXLL     0
V03MMXLA    50
V03MMXMM     0
V03MMXMA    50
dtype: int64

In [81]:
print('Number of NaN values per column: {}'.format(moaks_OA_features.isna().sum().to_list()))

Number of NaN values per column: [0, 0, 0, 0, 0, 0, 2, 10]


In [82]:
moaks_OA_features.fillna(-1.0,inplace=True)
moaks_OA_features

,ID,SIDE,XR_Osteophytes,FullTCL,AnyTCL,PartialTCL,BML,Meniscus_Degradation
0,9031930,1,1.0,0.0,1.0,1.0,0.0,1.0
1,9039627,1,1.0,0.0,1.0,1.0,0.0,-1.0
2,9055038,1,1.0,0.0,1.0,1.0,1.0,1.0
3,9056978,1,1.0,0.0,1.0,1.0,-1.0,-1.0
4,9057479,1,1.0,0.0,1.0,1.0,0.0,1.0
5,9057479,2,1.0,0.0,1.0,1.0,0.0,-1.0
6,9064631,2,1.0,0.0,1.0,1.0,0.0,1.0
7,9081858,2,1.0,1.0,1.0,0.0,1.0,-1.0
8,9082137,1,1.0,0.0,1.0,1.0,0.0,1.0
9,9086407,1,1.0,0.0,1.0,1.0,1.0,1.0


In [83]:
moaks_OA_features.to_csv('moaks_sq03_OA_features.csv')

# NOW WE CONSTRUCT THE DF THAT CONTAINS THE OA CRITERIA AS COLUMNS

In [84]:
moaks_OA_criteria = pd.DataFrame(columns=['ID','SIDE',
             'Osteo_AND_FullTCL',
             'Osteo_AND_PartialTCL_AND_BML',
             'Osteo_AND_PartialTCL_AND_Meniscus_Degradation',
             'Osteo_AND_BML_AND_Meniscus_Degradation',
             'FullTCL_AND_BML_AND_Meniscus_Degradation'])
# the ID and SIDE columns are integers
moaks_OA_criteria[['ID','SIDE']] = common_sq_kxr_03[['ID','SIDE']]
# and we fill the rest of the df with np.nan values
moaks_OA_criteria.iloc[:,2:] = np.nan
moaks_OA_criteria

,ID,SIDE,Osteo_AND_FullTCL,Osteo_AND_PartialTCL_AND_BML,Osteo_AND_PartialTCL_AND_Meniscus_Degradation,Osteo_AND_BML_AND_Meniscus_Degradation,FullTCL_AND_BML_AND_Meniscus_Degradation
0,9031930,1,NaN,NaN,NaN,NaN,NaN
1,9039627,1,NaN,NaN,NaN,NaN,NaN
2,9055038,1,NaN,NaN,NaN,NaN,NaN
3,9056978,1,NaN,NaN,NaN,NaN,NaN
4,9057479,1,NaN,NaN,NaN,NaN,NaN
5,9057479,2,NaN,NaN,NaN,NaN,NaN
6,9064631,2,NaN,NaN,NaN,NaN,NaN
7,9081858,2,NaN,NaN,NaN,NaN,NaN
8,9082137,1,NaN,NaN,NaN,NaN,NaN
9,9086407,1,NaN,NaN,NaN,NaN,NaN


In [85]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['FullTCL'] == 1.0)),'Osteo_AND_FullTCL'] = 1.0
         
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['FullTCL'] != 1.0)),'Osteo_AND_FullTCL'] = 0.0

print('With Definite Osteophytes AND FullTCL->1.0, Without->0.0: \n{}'.
        format(moaks_OA_criteria['Osteo_AND_FullTCL'].value_counts(dropna=False)))        

With Definite Osteophytes AND FullTCL->1.0, Without->0.0: 
0.0    44
1.0     6
Name: Osteo_AND_FullTCL, dtype: int64


In [86]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['PartialTCL'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0)),'Osteo_AND_PartialTCL_AND_BML'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['PartialTCL'] != 1.0) | 
            (moaks_OA_features['BML'] != 1.0)),'Osteo_AND_PartialTCL_AND_BML'] = 0.0
print('With Definite Osteophytes AND PartialTCL AND BML->1.0, Without->0.0 \n{}'.
        format(moaks_OA_criteria.Osteo_AND_PartialTCL_AND_BML.value_counts(dropna=False)))

With Definite Osteophytes AND PartialTCL AND BML->1.0, Without->0.0 
0.0    36
1.0    14
Name: Osteo_AND_PartialTCL_AND_BML, dtype: int64


In [87]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['PartialTCL'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'Osteo_AND_PartialTCL_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) | 
            (moaks_OA_features['PartialTCL'] != 1.0) | 
            (moaks_OA_features['Meniscus_Degradation'] != 1.0)),'Osteo_AND_PartialTCL_AND_Meniscus_Degradation'] = 0.0
print('With Osteophytes AND Partial TCL AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 

With Osteophytes AND Partial TCL AND Meniscus_Degradation->1.0, Without->0.0: 
1.0    32
0.0    18
Name: Osteo_AND_PartialTCL_AND_Meniscus_Degradation, dtype: int64


In [88]:
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'Osteo_AND_BML_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['XR_Osteophytes'] != 1.0) |
            (moaks_OA_features['BML'] != 1.0) |
            (moaks_OA_features['Meniscus_Degradation'] != 1.0)),'Osteo_AND_BML_AND_Meniscus_Degradation'] = 0.0            
print('With Osteophytes AND BML AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 

With Osteophytes AND BML AND Meniscus_Degradation->1.0, Without->0.0: 
0.0    31
1.0    19
Name: Osteo_AND_BML_AND_Meniscus_Degradation, dtype: int64


In [89]:
moaks_OA_criteria.loc[((moaks_OA_features['FullTCL'] == 1.0) & 
            (moaks_OA_features['BML'] == 1.0) & 
            (moaks_OA_features['Meniscus_Degradation'] == 1.0)),'FullTCL_AND_BML_AND_Meniscus_Degradation'] = 1.0
moaks_OA_criteria.loc[((moaks_OA_features['FullTCL'] != 1.0) | 
            (moaks_OA_features['BML'] != 1.0) | 
            (moaks_OA_features['Meniscus_Degradation'] == 0.0)),'FullTCL_AND_BML_AND_Meniscus_Degradation'] = 0.0
print('With FullTCL AND BML AND Meniscus_Degradation->1.0, Without->0.0: \n{}'.
        format((moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degradation'].value_counts(dropna=False)))) 


With FullTCL AND BML AND Meniscus_Degradation->1.0, Without->0.0: 
0.0    45
1.0     4
NaN     1
Name: FullTCL_AND_BML_AND_Meniscus_Degradation, dtype: int64


In [90]:
print('Number of subjects without OA for each criterion is: \n{}'.format(moaks_OA_criteria.isna().sum()))

Number of subjects without OA for each criterion is: 
ID                                               0
SIDE                                             0
Osteo_AND_FullTCL                                0
Osteo_AND_PartialTCL_AND_BML                     0
Osteo_AND_PartialTCL_AND_Meniscus_Degradation    0
Osteo_AND_BML_AND_Meniscus_Degradation           0
FullTCL_AND_BML_AND_Meniscus_Degradation         1
dtype: int64


In [91]:
moaks_OA_criteria.fillna(-1.0,inplace=True)

In [92]:
moaks_OA_criteria.to_csv('moaks_sq03_OA_criteria.csv')

In [93]:
# NOW WE FIND THOSE WHO SATISFY AT LEAST ONE OF THE OA CRITERIA
moaks_OA_criteria['moaks_OA'] = np.nan
moaks_OA_criteria.loc[((moaks_OA_criteria['Osteo_AND_FullTCL']==1.0) |
                (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_BML']==1.0) |
                (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation']==1.0) |
                (moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation']==1.0) |
                (moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degradation']==1.0)),'moaks_OA'] = 1.0
# moaks_OA_criteria.loc[((moaks_OA_criteria['Osteo_AND_FullTCL']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_BML']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_PartialTCL_AND_Meniscus_Degradation']==0.0) &
#                 (moaks_OA_criteria['Osteo_AND_BML_AND_Meniscus_Degradation']==0.0) &
#                 (moaks_OA_criteria['FullTCL_AND_BML_AND_Meniscus_Degra`dation']==0.0)),'moaks_OA'] = 0.0
moaks_OA_criteria.fillna(0.0,inplace=True)

print('Number of subjects that satisfy at least one OA criterion is : {}'.format((moaks_OA_criteria['moaks_OA']==1.0).sum()))

Number of subjects that satisfy at least one OA criterion is : 39


In [94]:
print(moaks_OA_criteria.shape[0],'\n',moaks_OA_criteria['moaks_OA'].value_counts())

50 
 1.0    39
0.0    11
Name: moaks_OA, dtype: int64


In [95]:
print(moaks_OA_criteria)

         ID  SIDE  Osteo_AND_FullTCL  Osteo_AND_PartialTCL_AND_BML  \
0   9031930     1                0.0                           0.0   
1   9039627     1                0.0                           0.0   
2   9055038     1                0.0                           1.0   
3   9056978     1                0.0                           0.0   
4   9057479     1                0.0                           0.0   
5   9057479     2                0.0                           0.0   
6   9064631     2                0.0                           0.0   
7   9081858     2                1.0                           0.0   
8   9082137     1                0.0                           0.0   
9   9086407     1                0.0                           1.0   
10  9101076     2                0.0                           0.0   
11  9120059     2                0.0                           1.0   
12  9135902     1                0.0                           1.0   
13  9135902     2   

# NOW WE CREATE THE DATAFRAME CONTAINING INFORMATION REGARDING 
# 1.MOAKS FEATURES 
# 2.MOAKS CRITERIA
# 3.MOAKS GRADES
# 4.KL GRADES
# 5.MOAKS-KL GRADES CROSSTABLE

In [96]:
# Here we upload the DataFrame with the KL grades
# and we merge the MOAKS and KL dataframes
kl_id_side_grade = pd.read_csv('/home/anastasis/EMC_Thesis/My_codes/KL_readings/KL_SQ03_ID_SIDE_GRADE.csv')
moaks_kl_OA_full = moaks_OA_criteria.merge(kl_id_side_grade,on=['ID','SIDE'],how='inner')
print(moaks_kl_OA_full.shape[0],'\n',moaks_kl_OA_full)

50 
          ID  SIDE  Osteo_AND_FullTCL  Osteo_AND_PartialTCL_AND_BML  \
0   9031930     1                0.0                           0.0   
1   9039627     1                0.0                           0.0   
2   9055038     1                0.0                           1.0   
3   9056978     1                0.0                           0.0   
4   9057479     1                0.0                           0.0   
5   9057479     2                0.0                           0.0   
6   9064631     2                0.0                           0.0   
7   9081858     2                1.0                           0.0   
8   9082137     1                0.0                           0.0   
9   9086407     1                0.0                           1.0   
10  9101076     2                0.0                           0.0   
11  9120059     2                0.0                           1.0   
12  9135902     1                0.0                           1.0   
13  9135902    

In [97]:
print('Number of different KL grades in the moaks_kl_OA_full dataframe:\n{}'
        .format(moaks_kl_OA_full['V03XRKL'].value_counts()))
moaks_kl_OA_full.to_csv('moaks_sq03_OA_KL_both_grades.csv')

Number of different KL grades in the moaks_kl_OA_full dataframe:
2.0    43
3.0     6
4.0     1
Name: V03XRKL, dtype: int64


In [98]:
moaks_KL_OA_all_info = moaks_OA_features.merge(moaks_kl_OA_full,on=['ID','SIDE'],how='outer',indicator=True)
print(moaks_KL_OA_all_info._merge.value_counts(),'\n',moaks_KL_OA_all_info.info())
moaks_KL_OA_all_info.to_csv('moaks_sq03_all_info.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype   
---  ------                                         --------------  -----   
 0   ID                                             50 non-null     int64   
 1   SIDE                                           50 non-null     int64   
 2   XR_Osteophytes                                 50 non-null     float64 
 3   FullTCL                                        50 non-null     float64 
 4   AnyTCL                                         50 non-null     float64 
 5   PartialTCL                                     50 non-null     float64 
 6   BML                                            50 non-null     float64 
 7   Meniscus_Degradation                           50 non-null     float64 
 8   Osteo_AND_FullTCL                              50 non-null     float64 
 9   Osteo_AND_PartialTCL_AND_BML                 

In [99]:
all_common_sq_kxr_03_sq03 = moaks_KL_OA_all_info.loc[:,['ID','SIDE','moaks_OA']]
print(all_common_sq_kxr_03_sq03.info())
all_moaks_vs_kl_grades_sq03 = moaks_KL_OA_all_info.loc[:,['ID','SIDE','moaks_OA','V03XRKL']]
print(all_moaks_vs_kl_grades_sq03.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        50 non-null     int64  
 1   SIDE      50 non-null     int64  
 2   moaks_OA  50 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.6 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        50 non-null     int64  
 1   SIDE      50 non-null     int64  
 2   moaks_OA  50 non-null     float64
 3   V03XRKL   50 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 2.0 KB
None


In [100]:
print('Sum of sq03 subjects that have MOAKS = 0.0: {}'.format((all_common_sq_kxr_03_sq03['moaks_OA']==0.0).sum()))
print('Sum of sq03 subjects that have MOAKS = 1.0: {}'.format((all_common_sq_kxr_03_sq03['moaks_OA']==1.0).sum()))

Sum of sq03 subjects that have MOAKS = 0.0: 11
Sum of sq03 subjects that have MOAKS = 1.0: 39


In [101]:
moaks_sq03_only_moaks_one = all_moaks_vs_kl_grades_sq03.loc[(all_moaks_vs_kl_grades_sq03['moaks_OA']==1.0)]
print(moaks_sq03_only_moaks_one,'\n','number of subjects with moaks grade = 1.0: ',moaks_sq03_only_moaks_one.moaks_OA.count())

         ID  SIDE  moaks_OA  V03XRKL
0   9031930     1       1.0      3.0
2   9055038     1       1.0      2.0
4   9057479     1       1.0      2.0
6   9064631     2       1.0      2.0
7   9081858     2       1.0      2.0
8   9082137     1       1.0      2.0
9   9086407     1       1.0      2.0
11  9120059     2       1.0      2.0
12  9135902     1       1.0      3.0
13  9135902     2       1.0      3.0
15  9179001     1       1.0      2.0
16  9205505     2       1.0      2.0
17  9223285     1       1.0      2.0
18  9255715     1       1.0      3.0
19  9281052     2       1.0      2.0
20  9281187     1       1.0      2.0
21  9297957     1       1.0      2.0
22  9305065     1       1.0      2.0
23  9331465     2       1.0      3.0
24  9381777     1       1.0      2.0
26  9446305     2       1.0      2.0
28  9486748     2       1.0      2.0
29  9559476     1       1.0      2.0
30  9615754     2       1.0      2.0
31  9624154     1       1.0      2.0
34  9702463     2       1.0      2.0
3

In [102]:
moaks_sq03_only_moaks_zero = all_moaks_vs_kl_grades_sq03.loc[(all_moaks_vs_kl_grades_sq03['moaks_OA']==0.0)]
print(moaks_sq03_only_moaks_zero,'\n','number of subjects with moaks grade = 0.0: ',moaks_sq03_only_moaks_zero.moaks_OA.count())

         ID  SIDE  moaks_OA  V03XRKL
1   9039627     1       0.0      2.0
3   9056978     1       0.0      2.0
5   9057479     2       0.0      2.0
10  9101076     2       0.0      2.0
14  9173950     2       0.0      2.0
25  9395235     2       0.0      2.0
27  9467722     2       0.0      2.0
32  9658152     1       0.0      2.0
33  9662112     1       0.0      2.0
37  9764947     2       0.0      2.0
42  9878804     1       0.0      2.0 
 number of subjects with moaks grade = 0.0:  11


In [103]:
# Here we save the dataframes containing the information we want
all_moaks_vs_kl_grades_sq03.to_csv('moaks_sq03_moaks_vs_kl.csv')
moaks_sq03_only_moaks_one.to_csv('moaks_sq03_only_moaks_one.csv')
moaks_sq03_only_moaks_zero.to_csv('moaks_sq03_only_moaks_zero.csv')
all_common_sq_kxr_03_sq03.to_csv('moaks_sq03_common_sq_kxr_03.csv')

In [104]:
moaks_OA_KL_crosstab = pd.crosstab(all_moaks_vs_kl_grades_sq03.moaks_OA,all_moaks_vs_kl_grades_sq03.V03XRKL,margins=True,margins_name='Total')
moaks_OA_KL_crosstab.to_csv('moaks_sq03_OA_KL_crosstab.csv')
moaks_OA_KL_crosstab


V03XRKL,2.0,3.0,4.0,Total
moaks_OA,,,,
0.0,11,0,0,11
1.0,32,6,1,39
Total,43,6,1,50


In [105]:
# NOW WE SAVE THE SUBJECTS WITH ZERO FEATURES IN ORDER TO INVESTIGATE THE INCIDENCE OA
moaks_OA_osteo_zero = moaks_OA_features.loc[moaks_OA_features.XR_Osteophytes == 0.0]
moaks_OA_osteo_zero.to_csv('moaks_sq03_Zero_Osteo.csv')
print('Number of subjects without definite osteophytes in SQ03 is : {}'.format(moaks_OA_osteo_zero.shape[0]))

moaks_OA_TCL_zero = moaks_OA_features.loc[(moaks_OA_features.FullTCL == 0.0) & (moaks_OA_features.PartialTCL == 0.0)]
moaks_OA_TCL_zero.to_csv('moaks_sq03_Zero_TCL.csv')
print('Number of subjects without TCL in SQ03 is : {}'.format(moaks_OA_TCL_zero.shape[0]))

moaks_OA_bml_zero = moaks_OA_features.loc[moaks_OA_features.BML == 0.0]
moaks_OA_bml_zero.to_csv('moaks_sq03_Zero_BML.csv')
print('Number of subjects without BML in SQ03 is : {}'.format(moaks_OA_bml_zero.shape[0]))

moaks_OA_meniscus_zero = moaks_OA_features.loc[moaks_OA_features.Meniscus_Degradation == 0.0]
moaks_OA_meniscus_zero.to_csv('moaks_sq03_Zero_Meniscus.csv')
print('Number of subjects without Meniscus Degradation in SQ03 is : {}'.format(moaks_OA_meniscus_zero.shape[0]))


Number of subjects without definite osteophytes in SQ03 is : 2
Number of subjects without TCL in SQ03 is : 2
Number of subjects without BML in SQ03 is : 28
Number of subjects without Meniscus Degradation in SQ03 is : 0
